In [1]:
import pandas as pd
import numpy as np
from glob import glob
from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_nemenyi_friedman, posthoc_nemenyi

In [2]:
result_type = 'Regression'
no_algos = 11 if result_type == 'Classification' else 13

In [3]:
rf_df = []
for result_file in glob('results/aggregated/*.csv'):
    if result_type in result_file:
        rf_df.append(pd.read_csv(result_file))
df = pd.concat(rf_df).drop('Unnamed: 0', axis = 1)

In [4]:
aggregated_results = '$' + df.groupby(['dataset', 'base', 'method']).mean()['PM1'].round(2).astype(str) + '\pm' + df.groupby(['dataset', 'base', 'method']).std()['PM1'].round(2).astype(str) + '$'
aggregated_results.unstack(level=0).to_csv('results/all_' + result_type + '.csv')

In [5]:
time_results = df.groupby(['dataset', 'base', 'method']).mean().groupby(['base','method']).mean()[['TriesToOpt','Time']]
time_results['MinTime'] = df.groupby(['dataset', 'base', 'method']).mean().groupby(['base','method']).min()[['Time']]
time_results['MaxTime'] = df.groupby(['dataset', 'base', 'method']).mean().groupby(['base','method']).max()[['Time']]
time_results.to_csv('results/time_' + result_type + '.csv')

In [6]:
datasets = df['dataset'].drop_duplicates().to_list()
for dataset in datasets:
    df_for_tests = df[['dataset','base','method']].drop_duplicates()
    temp_data_for_tests = []
    for i, row in df_for_tests.iterrows():
        if row.dataset == dataset:
            temp_data_for_tests.append(list(df[(df.dataset==row.dataset) & (df.base==row.base) & (df.method==row.method)]['PM1']))
    print(dataset, friedmanchisquare(*temp_data_for_tests))

abalone FriedmanchisquareResult(statistic=1237.118794835008, pvalue=1.8210328822666697e-256)
bike FriedmanchisquareResult(statistic=1175.982097976102, pvalue=2.600913655271004e-243)
boston FriedmanchisquareResult(statistic=810.1248128907287, pvalue=9.369268661230679e-165)
diabetes FriedmanchisquareResult(statistic=567.9626086000221, pvalue=5.1383522627383655e-113)
fire FriedmanchisquareResult(statistic=844.4273530575468, pvalue=4.186191883750187e-172)
machine FriedmanchisquareResult(statistic=584.6937285698484, pvalue=1.4022049999662132e-116)
student FriedmanchisquareResult(statistic=848.0294098208976, pvalue=7.075964263463571e-173)


In [7]:
rf_df1 = []
rf_df2 = []
for i, result_file in enumerate(glob('results/aggregated/*.csv')):
    if result_type in result_file:
        if i > (len(glob('results/aggregated/*.csv'))/2):
            rf_df1.append(pd.read_csv(result_file))
        else:
            rf_df2.append(pd.read_csv(result_file))

df1 = pd.concat(rf_df1).drop('Unnamed: 0', axis = 1)
df2 = pd.concat(rf_df2).drop('Unnamed: 0', axis = 1)

In [8]:
data_for_nemenyi_test = df1.groupby(['dataset', 'base', 'method']).mean()['PM1'].unstack(level=0).join(df2.groupby(['dataset', 'base', 'method']).mean()['PM1'].unstack(level=0), lsuffix='_')
data_for_nemenyi_test

dataset                                   abalone_         bike_    boston_  \
base    method                                                                
000 rf  000 all                           4.764343  4.508402e+05  11.175558   
001 knn 000 all                           5.917103  8.043019e+05  26.192931   
        002 intercorrelation              6.273205  9.139209e+05  26.073213   
        003 feature importance selection  5.422282  7.080720e+05  20.044577   
        004 l1                            6.218799  8.169518e+05  20.139768   
        005 relieff selection             5.566142  1.825072e+06  19.987322   
        006 forward selection             5.544419  1.303517e+06  21.739054   
        007 backward selection            5.541839  1.169805e+06  20.080230   
        008 stepwise selection            5.543897  1.182393e+06  20.150008   
        009 bayesian selection            5.531400  7.914195e+05  19.562863   
        010 correlation weighting         5.683070  6.208102e+05  20.586426   
        011 feature importance weighting  5.557096  8.155585e+05  19.976539   
        012 relieff weighting             5.543361  1.157508e+06  20.018558   
        013 bayesian weighting            5.233078  5.545236e+05  15.985604   

dataset                                     diabetes_     fire_      machine_  \
base    method                                                                  
000 rf  000 all                           3366.645782  2.215851   4476.904780   
001 knn 000 all                           3351.298596  2.042398  10568.905302   
        002 intercorrelation              3331.938500  2.034876  10733.434979   
        003 feature importance selection  3801.438423  2.109935  10013.722006   
        004 l1                            3245.766551 -1.000000   9113.271263   
        005 relieff selection             3250.884875  2.044705  11239.483200   
        006 forward selection             3424.378642  2.077612  10361.290293   
        007 backward selection            3270.026993  2.077293  10248.455171   
        008 stepwise selection            3314.302524  2.076431  10131.789318   
        009 bayesian selection            3181.231185  1.937094   7759.183442   
        010 correlation weighting         3246.631623  2.052402   9500.974505   
        011 feature importance weighting  3245.629297  2.049432   9142.617740   
        012 relieff weighting             3251.573938  2.065308  10547.970100   
        013 bayesian weighting            2880.369426  1.870549   6711.558152   

dataset                                    student_   abalone          bike  \
base    method                                                                
000 rf  000 all                           15.252728  4.765347  4.460808e+05   
001 knn 000 all                           19.295810  5.872726  8.245619e+05   
        002 intercorrelation              19.295810  6.208608  9.205672e+05   
        003 feature importance selection  21.540813  5.394071  7.403873e+05   
        004 l1                            19.541044  6.174924  8.490095e+05   
        005 relieff selection             18.547160  5.518201  1.786537e+06   
        006 forward selection             19.705728  5.505612  1.250821e+06   
        007 backward selection            18.249218  5.495499  1.205888e+06   
        008 stepwise selection            19.079174  5.496323  1.217179e+06   
        009 bayesian selection            16.404120  5.484877  8.206247e+05   
        010 correlation weighting         18.768985  5.644057  6.258672e+05   
        011 feature importance weighting  18.543548  5.509009  8.471582e+05   
        012 relieff weighting             18.173847  5.497148  1.190827e+06   
        013 bayesian weighting            14.886868  5.197224  5.568685e+05   

dataset                                      boston     diabetes      fire  \
base    method                                                               
000 rf  000 all  

In [9]:
data_for_nemenyi_test = df.groupby(['dataset', 'base', 'method']).mean()['PM1'].unstack(level=0)
data_for_nemenyi_test['ne_fri_test'] = list(posthoc_nemenyi_friedman(data_for_nemenyi_test.to_numpy().T)[no_algos])
data_for_nemenyi_test['ne_fri_test_bool'] = data_for_nemenyi_test['ne_fri_test'] < 0.05
data_for_nemenyi_test[['ne_fri_test', 'ne_fri_test_bool']]

dataset                                   ne_fri_test  ne_fri_test_bool
base    method                                                         
000 rf  000 all                              0.900000             False
001 knn 000 all                              0.010668              True
        002 intercorrelation                 0.006537              True
        003 feature importance selection     0.036807              True
        004 l1                               0.506161             False
        005 relieff selection                0.081451             False
        006 forward selection                0.001159              True
        007 backward selection               0.327854             False
        008 stepwise selection               0.045216              True
        009 bayesian selection               0.900000             False
        010 correlation weighting            0.327854             False
        011 feature importance weighting     0.791954             False
        012 relieff weighting                0.371999             False
        013 bayesian weighting               1.000000             False

In [10]:
data_for_nemenyi_test = df.groupby(['dataset', 'base', 'method']).mean()['PM1'].unstack(level=0)
data_for_nemenyi_test['repl_d1'] = data_for_nemenyi_test['abalone']
data_for_nemenyi_test['repl_d1'] = data_for_nemenyi_test['bike']
data_for_nemenyi_test['repl_d1'] = data_for_nemenyi_test['boston']
data_for_nemenyi_test['repl_d1'] = data_for_nemenyi_test['diabetes']
data_for_nemenyi_test['repl_d1'] = data_for_nemenyi_test['fire']
data_for_nemenyi_test['repl_d1'] = data_for_nemenyi_test['machine']
data_for_nemenyi_test['ne_fri_test'] = list(posthoc_nemenyi_friedman(data_for_nemenyi_test.to_numpy().T)[no_algos])
data_for_nemenyi_test['ne_fri_test_bool'] = data_for_nemenyi_test['ne_fri_test'] < 0.05
data_for_nemenyi_test[['ne_fri_test', 'ne_fri_test_bool']]

dataset                                   ne_fri_test  ne_fri_test_bool
base    method                                                         
000 rf  000 all                              0.900000             False
001 knn 000 all                              0.002632              True
        002 intercorrelation                 0.001218              True
        003 feature importance selection     0.018864              True
        004 l1                               0.527344             False
        005 relieff selection                0.015223              True
        006 forward selection                0.001000              True
        007 backward selection               0.251056             False
        008 stepwise selection               0.028538              True
        009 bayesian selection               0.900000             False
        010 correlation weighting            0.285370             False
        011 feature importance weighting     0.756485             False
        012 relieff weighting                0.165830             False
        013 bayesian weighting               1.000000             False